# Setup

## 1. Docker Image 저장 경로 변경

Change Docker image path to EBS
- SageMaker 노트북 인스턴스일 경우 docker image 저장소의 공간이 작아 마운트한 EBS로 변경을 진행합니다.
SageMaker 노트북 인스턴스에서 로컬 모드 디버깅 시 종종 No space left 관련 오류가 발생합니다. 따라서, 도커 이미지/컨테이너가 저장될 폴더를 SageMaker EBS (Amazon Elastic Block Store) 볼륨으로 변경하는 것을 권장합니다. 도커 이미지/컨테이너는 기본적으로 EBS가 아닌 루트 볼륨에 저장하기 때문에(루트 볼륨의 크기는 사용자가 임의로 조정할 수 없습니다!) 고용량의 이미지들을 빌드하면 용량이 꽉 차기 때문입니다.

In [1]:
%%bash

#!/usr/bin/env bash

echo '{
    "runtimes": {
        "nvidia": {
            "path": "nvidia-container-runtime",
            "runtimeArgs": []
        }
    }
}' > daemon.json

sudo cp daemon.json /etc/docker/daemon.json && rm daemon.json

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

sudo docker info | grep Root

Add data-root and default-shm-size=10G


Redirecting to /bin/systemctl restart docker.service


Docker Restart
 Docker Root Dir: /home/ec2-user/SageMaker/.container/docker


## 2. 패키지 설치

In [2]:
import os
requirement_path = os.path.join(os.getcwd(), "requirements.txt")
!pip install -r $requirement_path

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.5 MB/s eta 0:00:00
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 kB 18.2 MB/s eta 0:00:00a 0:00:01
Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)


## 3. Jupyter Server 재실행
#### [중요] 아래를 실행하면 모든 노트북의 커널이 중지됩니다. 유의해서 사용 하세요.

In [ ]:
import os
os.system('sudo systemctl restart jupyter-server')